### Import important libraries

In [2]:
import torchreid
import numpy as np
import sklearn.model_selection
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import torch

from torchreid.models import build_model
import torch.nn as nn
from torchreid.utils import FeatureExtractor
from torch.nn import functional as F
from torchreid import metrics

import glob
import os.path as osp

from torchreid.data import ImageDataset
torchreid.models.show_avai_models()

c:\Users\User\anaconda3\envs\torchreid2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [3]:
#Check if GPU is being used

torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.get_device_name(0)
torch. __version__

'1.12.1+cu113'

### Register Occluded-Reid into the dataset list

In [4]:
from torchreid.data import ImageDataset
from sklearn.model_selection import train_test_split

class OccludedREID(ImageDataset):
    dataset_dir = 'occluded_reid'

    def __init__(self, root='', split_ratio=0.5, **kwargs):
        self.root = osp.abspath(osp.expanduser(root))
        self.occluded_dir = osp.join(self.root, self.dataset_dir, 'occluded_body_images')
        self.whole_dir = osp.join(self.root, self.dataset_dir, 'whole_body_images')

        pid_container = self._get_pid_container(self.whole_dir)
        train_pids, test_pids = train_test_split(list(pid_container), test_size=split_ratio, random_state=42)

        train = self._process_dir(self.whole_dir, train_pids, relabel=True)
        gallery = self._process_dir(self.whole_dir, test_pids, relabel=False)
        query = self._process_dir(self.occluded_dir, test_pids, relabel=False)

        super(OccludedREID, self).__init__(train, query, gallery, **kwargs)

    def _get_pid_container(self, dir_path):
        img_paths = glob.glob(osp.join(dir_path, '*', '*.tif'))

        pid_container = set()
        for img_path in img_paths:
            pid = int(osp.basename(osp.dirname(img_path)))
            pid_container.add(pid)

        return pid_container

    def _process_dir(self, dir_path, pids, relabel=False):
        img_paths = glob.glob(osp.join(dir_path, '*', '*.tif'))

        pid2label = {pid: label for label, pid in enumerate(pids)}

        dataset = []
        for img_path in img_paths:
            pid = int(osp.basename(osp.dirname(img_path)))
            if pid not in pids: continue
            if relabel: pid = pid2label[pid]
            img_name = osp.basename(img_path)
            camid = int(img_name.split('_')[1].split('.')[0])
            dataset.append((img_path, pid, camid))

        return dataset

# Register your dataset
import torchreid
torchreid.data.register_image_dataset('occluded_reid', OccludedREID)

## Person Re-identification

### Market-1501 Person Re-identification

#### Softmax Loss

Change the model name based on what models you would like to use. Refer to the top part for the list of models available. In addition, change the save directory towards what you want also.

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)


model = torchreid.models.build_model(
    name="osnet_x0_75",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir="log/osnet_x0_75_market1501softmax_normalized_310523",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    normalize_feature= True
)

#### Triplet Loss

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    height=256,
    width=128,
    combineall=False,
    batch_size=32,
    num_instances=4,
    train_sampler='RandomIdentitySampler' # this is important
)

model = torchreid.models.build_model(
    name='osnet_x0_75',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)
engine.run(
    max_epoch=60,
    save_dir='log/osnet_x0_75_market1501triplet_normalized_310523',
    print_freq=10,
    normalize_feature= True
)

### Occluded-Reid Person Re-identification

#### Softmax Loss

Again, change the model name based on what you would like to use.

In [ ]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir="log/resnet50_market1501triplet_papersetting_090623",
    max_epoch=100,
    eval_freq=10,
    print_freq=10,
    test_only=False,
    normalize_feature= True
)

#### Triplet Loss

In [ ]:
model = torchreid.models.build_model(
    name='resnet50mid',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003,
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=0.7, weight_x=1, scheduler=scheduler
)
engine.run(
    max_epoch=100,
    save_dir='log/resnet50mid_forgettingbest',
    normalize_feature= True
)

## Ensemble Learning Person Re-identification

### Market-1501 Ensemble

In [ ]:
# Extract features from the testing data
test_loaders = datamanager.test_loader

query_loader = test_loaders['market1501']['query']
gallery_loader = test_loaders['market1501']['gallery']


#Define extractor
extractor1 = FeatureExtractor(model_path='log/osnet_x1_0_market1501final/model/model.pth.tar-100', model_name='osnet_x1_0', device='cuda')
use_gpu = torch.cuda.is_available()

extractor2  = FeatureExtractor(model_path='log/osnet_x0_75_market1501final/model/model.pth.tar-100', model_name='osnet_x0_75', device='cuda')

@torch.no_grad()
def feature_extraction(data_loader, extractor, use_gpu):
    f_, pids_, camids_ = [], [], []
    for batch_idx, data in enumerate(data_loader):
        imgs = data['img']
        pids = data['pid']
        camids = data['camid']
        if use_gpu:
            imgs = imgs.cuda()
        features = extractor(imgs)  # use the extractor instance directly
        features = features.cpu()
        f_.append(features)
        pids_.extend(pids.tolist())
        camids_.extend(camids.tolist())
    f_ = torch.cat(f_, 0)
    pids_ = np.asarray(pids_)
    camids_ = np.asarray(camids_)
    return f_, pids_, camids_

#Extracting features using extractor1
print('Extracting features from query set ...')
qf, q_pids, q_camids = feature_extraction(query_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf, g_pids, g_camids = feature_extraction(gallery_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))

#Extracting features using extractor2
print('Extracting features from query set ...')
qf2, q_pids2, q_camids2 = feature_extraction(query_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf2, g_pids2, g_camids2 = feature_extraction(gallery_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))

qf_avg = (qf + qf2  ) / 2
gf_avg = (gf + gf2 ) / 2

#Compute distance matrix for extractor1
print('Computing distance matrix with metric={} ...'.format("euclidean"))
distmat = metrics.compute_distance_matrix(qf_avg, gf_avg, metric="euclidean")
distmat = distmat.numpy()



print('Computing CMC and mAP ...')
cmc, mAP = metrics.evaluate_rank(
    distmat,
    q_pids,
    g_pids,
    q_camids,
    g_camids,
)

# print results
print(f"mAP: {mAP:.1%}, Rank-1: {cmc[0]:.1%}, Rank-5: {cmc[4]:.1%}, Rank-10: {cmc[9]:.1%},Rank-20: {cmc[19]:.1%}")

### Occluded-Reid Ensemble

In [ ]:
from torchreid.models import build_model
import torch.nn as nn
from torchreid.utils import FeatureExtractor
from torch.nn import functional as F
import numpy as np
from torchreid import metrics

# Extract features from the testing data
test_loaders = datamanager.test_loader

query_loader = test_loaders['occluded_reid']['query']
gallery_loader = test_loaders['occluded_reid']['gallery']


#Define extractor
extractor1 = FeatureExtractor(model_path='log/densenet169_ocreidfinal/model/model.pth.tar-100', model_name='densenet169', device='cuda')
use_gpu = torch.cuda.is_available()

extractor2  = FeatureExtractor(model_path='log/densenet161_ocreidfinal/model/model.pth.tar-100', model_name='densenet161', device='cuda')

@torch.no_grad()
def feature_extraction(data_loader, extractor, use_gpu):
    f_, pids_, camids_ = [], [], []
    for batch_idx, data in enumerate(data_loader):
        imgs = data['img']
        pids = data['pid']
        camids = data['camid']
        if use_gpu:
            imgs = imgs.cuda()
        features = extractor(imgs)  # use the extractor instance directly
        features = features.cpu()
        f_.append(features)
        pids_.extend(pids.tolist())
        camids_.extend(camids.tolist())
    f_ = torch.cat(f_, 0)
    pids_ = np.asarray(pids_)
    camids_ = np.asarray(camids_)
    return f_, pids_, camids_

#Extracting features using extractor1
print('Extracting features from query set ...')
qf, q_pids, q_camids = feature_extraction(query_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf, g_pids, g_camids = feature_extraction(gallery_loader, extractor1, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))



#Extracting features using extractor2
print('Extracting features from query set ...')
qf2, q_pids2, q_camids2 = feature_extraction(query_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(qf.size(0), qf.size(1)))

print('Extracting features from gallery set ...')
gf2, g_pids2, g_camids2 = feature_extraction(gallery_loader, extractor2, use_gpu)
print('Done, obtained {}-by-{} matrix'.format(gf.size(0), gf.size(1)))


qf_avg = (qf + qf2  ) / 2
gf_avg = (gf + gf2 ) / 2

#Compute distance matrix for extractor1
print('Computing distance matrix with metric={} ...'.format("euclidean"))
distmat = metrics.compute_distance_matrix(qf_avg, gf_avg, metric="euclidean")
distmat = distmat.numpy()



print('Computing CMC and mAP ...')
cmc, mAP = metrics.evaluate_rank(
    distmat,
    q_pids,
    g_pids,
    q_camids,
    g_camids,
)

print("This is cmc",cmc)
# print results
print(f"mAP: {mAP:.1%}, Rank-1: {cmc[0]:.1%}, Rank-5: {cmc[4]:.1%}, Rank-10: {cmc[9]:.1%}, Rank-20: {cmc[19]:.1%}")